<a href="https://colab.research.google.com/github/AppleTater/Jinkerson-Lab-Research/blob/master/Cumulative.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Files that need to be uploaded


---
* TAGData.csv
* large-lib_full_2016.10.25.csv




# Importing Libraries


**Python Libraries**


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

import numpy.matlib
import numpy as np
from numpy import median

import scipy.stats as stats
import scipy.special

import re
import sys

from IPython.display import (
    display_pretty, display_html, display_jpeg,
    display_png, display_json, display_latex, display_svg
)

from IPython.display import display
import itertools
import gc

import statsmodels.api as sm
import statsmodels.stats.multitest as smm
import statsmodels.formula.api as smf

**R Libraries**

In [ ]:
%reload_ext rpy2.ipython

# %%R
# library(ggplot2)

import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
from rpy2.robjects.vectors import FloatVector
import gc
R_stats = importr('stats')

from rpy2.rinterface_lib.sexp import StrSexpVector
from functools import partial
from rpy2.ipython import html
html.html_rdataframe=partial(html.html_rdataframe, table_class="docutils")

from rpy2.robjects.conversion import localconverter

import pandas as pd
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri

from rpy2.robjects.conversion import localconverter

# Making of the Merged File



In [ ]:
data = pd.read_csv('TAG Data.csv')

data = data[['gene' , 'R5-S#11-HLG_WHOLE_normalized_reads' , 'R5-S#11-ALL_WHOLE_normalized_reads']]
data ['Ratios: HLG / ALL (Whole)'] = data ['R5-S#11-HLG_WHOLE_normalized_reads'] / data['R5-S#11-ALL_WHOLE_normalized_reads']

data = data.sort_values('gene')
data = data.replace([np.inf, -np.inf], np.nan)
data = data.dropna()

data = data.drop(['R5-S#11-HLG_WHOLE_normalized_reads' , 'R5-S#11-ALL_WHOLE_normalized_reads'], axis = 1)

data = pd.DataFrame(data)

mean = data.groupby(['gene']).mean()
mean.rename(columns = {'Ratios: HLG / ALL (Whole)': 'Ratio Mean'}, inplace = True)
# mean

median = data.groupby(['gene']).median()
median.rename(columns = {'Ratios: HLG / ALL (Whole)': 'Ratio Median'}, inplace = True)
# median

count = data.groupby(['gene']).count()
count.rename(columns = {'Ratios: HLG / ALL (Whole)': 'Count'}, inplace = True)
# count

result = data.reset_index(drop = True)
result = pd.concat([mean, median, count], axis = 1)

result = result.sort_values(by = ['Ratio Median'], ascending = False)

counter = float(input("Enter the count number that you want results that are less than to be cut off at: "))
greater = result[result['Count'] < counter].index

result.drop(greater, inplace = True)
result.reset_index(inplace = True)

groupby = result.rename(columns = {'gene' : "Gene"})

#Confidence Level
ba = pd.read_csv('large-lib_full_2016.10.25.csv')

ba = ba.drop(['well', 'side', 'chromosome', 'plate', 'strand', 
                  'min_position', 'full_position', 'orientation', 
                  'gene_end_distances', 'total_reads', 
                  'if_both_sides', 'LEAPseq_distance', 
                  'LEAPseq_percent_confirming',
                  'LEAPseq_N_confirming', 'LEAPseq_N_nonconf', 
                  'LEAPseq_N_unique_pos',
                  'flanking_seq', 'plate_errors', 'well_errors', 
                  'plate_rpm', 'well_rpm',
                  'if_fixed_position'], axis = 1)

level = float(input("Enter the confidence level that you want the data to be less than or equal to: "))
filterba = ba[ba['confidence_level'] <= level]
filterba = filterba.sort_values('confidence_level')
filterba = filterba.rename(columns = {"gene" : "Gene",
                                          "confidence_level" : "Confidence Level",
                                          "feature" : "Feature"})
filterba = filterba.reset_index(drop = True)

genie = filterba["Gene"].astype(str).str.replace(r'.v5.5$', '')
genie = genie.str.replace(r".v5.5", '').str.strip()

filterba = filterba.drop(columns = {"Gene"})
filterba["Gene"] = genie

filterba =  filterba[["Gene", "Confidence Level", "IB", "Feature"]]

#GO Term
#gogo = pd.read_csv('GOTerms.csv')

#Merging all 3
df1 = groupby
#df2 = gogo
df3 = filterba

leftBigMerge = pd.merge(left = df1, right = df3, how = 'left', 
                        left_on = 'Gene', right_on = 'Gene')
#leftBigMerge = pd.merge(left = leftBigMerge, right = df3, how = 'left',
                        #left_on = 'Gene', right_on = 'Gene')
#leftBigMerge = leftBigMerge.drop(columns = ["Unnamed: 0_x", "Unnamed: 0_y", 
                                   # "Unnamed: 0"])
leftBigMerge = leftBigMerge[["Gene", "Ratio Median", "Ratio Mean", #"GO Term", 
                             "Feature", "Confidence Level", "IB"]]

Enter the count number that you want results that are less than to be cut off at: 3


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Enter the confidence level that you want the data to be less than or equal to: 3


# Fishing 🐟🐠

**Fisher Exact Test Preparation**

In [ ]:
#matrix = df.to_numpy()
matrix = leftBigMerge

oarfish = pd.DataFrame(matrix, columns = ["Gene", "Ratio Median",
                                          "Ratio Mean", #"GO Term",
                                          "Description"])

oarfish = oarfish.drop_duplicates()

data = pd.read_csv('TAG Data.csv')

data = data[['IB' , 'gene' , 'R5-S#11-HLG_WHOLE_normalized_reads' , 'R5-S#11-ALL_WHOLE_normalized_reads']]
data ['Ratios: HLG / ALL (Whole)'] = data ['R5-S#11-HLG_WHOLE_normalized_reads'] / data['R5-S#11-ALL_WHOLE_normalized_reads']

data = data.sort_values('gene')
data = data.replace([np.inf, -np.inf], np.nan)
data = data.dropna()
data = data.drop(columns = {"R5-S#11-HLG_WHOLE_normalized_reads", 
                            "R5-S#11-ALL_WHOLE_normalized_reads"})

ratio = float(input("Enter value that you want data to be greater than (Ratio: HLG / ALL (Whole)): "))

data["RatioType"] = np.where(data["Ratios: HLG / ALL (Whole)"] > ratio, 
                                 "Hit", "Non-Hit")
data = data.sort_values("gene")
data = data.reset_index(drop = True)
data = data.replace(['unknown_chrom', 'no_gene_found', 
                     'gene_unknown'], np.nan)
data = data.dropna()

unknown = data[data.gene.eq("unknown_chrom") |
               data.gene.eq("no_gene_found") |
               data.gene.eq("gene_unknown")]

unknown = unknown.reset_index(drop = True)

for gene, data_gene in data.groupby('gene'):
    data_gene

data_gene.iloc[-1]

Enter value that you want data to be greater than (Ratio: HLG / ALL (Whole)): 5


IB                           TGTTTTCGTGCTCCTACGACAG
gene                                  Cre52.g761647
Ratios: HLG / ALL (Whole)                         0
RatioType                                   Non-Hit
Name: 103055, dtype: object

**Actual Fisher Exact Test**

In [ ]:
bigHit = 0
bigNo = 0

lastGene = data_gene['gene'].iloc[-1]
count = float(input("For the total number of counts (hits + non-hits) of a gene, enter a number for cutoff : "))

for gene, data_genie in data.groupby('gene'):
    data_genie = data_genie.drop(columns = {'IB', "Ratios: HLG / ALL (Whole)"})
   
    hit = data_genie.loc[data_genie.RatioType == 'Hit', "RatioType"].count()
    noHit = data_genie.loc[data_genie.RatioType == "Non-Hit", "RatioType"].count()

    bigHit =  bigHit + hit
    bigNo = bigNo + noHit
    if lastGene in data_genie.values:
        sum = pd.DataFrame (
            { 
#                 'Gene': "  ",
                'Hit' : bigHit,
                'Non-Hit' : bigNo
#                 'Total': bigTotal
            }, index = [0]
        )

#         sum = sum.to_numpy()

goaway = pd.DataFrame()
biggergoaway = pd.DataFrame()

goname = pd.DataFrame()
biggergoname = pd.DataFrame()

for gene, data_genie in data.groupby('gene'):
    data_genie = data_genie.drop(columns = {'IB', "Ratios: HLG / ALL (Whole)"})
   
    hit = data_genie.loc[data_genie.RatioType == 'Hit', "RatioType"].count()
    noHit = data_genie.loc[data_genie.RatioType == "Non-Hit", "RatioType"].count()
#     total = hit + noHit
    
    bigHit =  bigHit + hit
    bigNo = bigNo + noHit
#     bigTotal = bigTotal + total
    
    genius = pd.DataFrame (
        {
        'Gene': data_genie.gene.unique()
        }
    )
    
    frame = pd.DataFrame ( 
         {
#             'Gene': data_genie.gene.unique(),
            'Hit' : hit,
            'Non-Hit' : noHit
#             'Total' : total
        }, index = [0]
    )

    concat = pd.concat([frame, sum], ignore_index = True)  
    concat = concat.to_numpy()
    
    obs = np.array(concat)
    new = pd.DataFrame(obs)

    oddsratio, fisher = stats.fisher_exact(new)
    
    fishy = np.asarray(fisher)
    fishy = fishy.reshape(-1, 1)

    frame.astype(float)
    sum.astype(float)

    fsh = pd.DataFrame(data = fishy, columns = ["p-value"])
    fsh.astype(float)
    
    oarfish = np.concatenate((frame, sum), axis = 0)
    oarfish = pd.DataFrame(data = oarfish, columns = ["Hit", "Non-Hit"])

    anchovy = pd.DataFrame(data = fsh, columns = ["p-value"])

    bigFish = pd.concat([oarfish, anchovy], ignore_index = True, sort = True)
    bigFish = pd.concat([genius, bigFish], axis = 1)

    bigFish.loc[bigFish['Hit'] + bigFish['Non-Hit'] < count] = "Go Away"

    if bigFish.iloc[0, 0] == "Go Away":
      bigFish.iloc[2, 3] = "NO"

    goaway = bigFish.iloc[2]
    biggergoaway = biggergoaway.append(goaway)

    goname = bigFish.iloc[0]
    biggergoname = biggergoname.append(goname)

For the total number of counts (hits + non-hits) of a gene, enter a number for cutoff : 12


In [ ]:
away = biggergoaway.drop(columns = ["Gene", "Hit", "Non-Hit"])
away = away[away["p-value"] != "NO"]
away = away.reset_index(drop = True)


pen = biggergoname.drop(columns = ["Hit", "Non-Hit", "p-value"])
pen = pen[pen["Gene"] != "Go Away"]
pen = pen.reset_index(drop = True)


pen_away = pen.merge(away, left_index = True, right_index = True)

#Benjamini-Hochberg Adjustment

In [ ]:
penguin = pen_away[~pen_away.Gene.str.contains(" & ")]
penguin = penguin.reset_index(drop = True)

bh_num = float(input("Enter the value that all adjusted p-values should be less than (0-1): "))

def FDR_adjust_pvalues(case, N=None, method='BH'):
    """ Adjust a list of p-values for false discovery rate using R's stats::p.adjust function.

    N and method are passed to R_stats.p_adjust:
     - N is the number of comparisons (if left unspecified, defaults to len(pvalue_list), I think)
     - method is the name of the adjustment method to use (inherited from R)

    Note that this MUST be done after all the p-values are already collected, on the full list of p-values at once:
     trying to do it on single p-values, even with adjusted N, will give different results!
    """
    if not method in R_stats.p_adjust_methods:
        raise ValueError("Unknown method %s - method must be one of (%s)!"%(method, ', '.join(R_stats.p_adjust_methods)))
    if N is None:   return R_stats.p_adjust(FloatVector(case), method=method)
    else:           return R_stats.p_adjust(FloatVector(case), method=method, n=N)

# example use
penguin["Adjusted p-value"] = FDR_adjust_pvalues(penguin['p-value'])

penguin.sort_values(by = 'p-value')

adelie = penguin.loc[penguin["Adjusted p-value"] < bh_num]

display(adelie)

#adelie.to_csv("Less than 1 p-values.csv")

Enter the value that all adjusted p-values should be less than (0-1): 0.2


,Gene,p-value,Adjusted p-value
1093,Cre13.g589400,0.000115968,0.157717
